In [120]:
import numpy as np
from scipy.stats import f

class TwoWayAnova:
    def __init__(self):
        self.results = {}

    def fit(self, data, value_col, factor_a, factor_b):
        self.data = data
        self.value_col = value_col
        self.factor_a = factor_a
        self.factor_b = factor_b

        a_levels = data[factor_a].unique()
        b_levels = data[factor_b].unique()
        grand_mean = data[value_col].mean()

        dfA = len(a_levels) - 1
        dfB = len(b_levels) - 1
        dfAB = dfA * dfB
        dfW = len(data) - (len(a_levels) * len(b_levels))

        # SS for Factor A
        SSA = 0
        for a in a_levels:
            group = data[data[factor_a] == a][value_col]
            SSA += len(group) / len(b_levels) * (group.mean() - grand_mean) ** 2

        # SS for Factor B
        SSB = 0
        for b in b_levels:
            group = data[data[factor_b] == b][value_col]
            SSB += len(group) / len(a_levels) * (group.mean() - grand_mean) ** 2

        # SS for Interaction
        SSAB = 0
        for a in a_levels:
            for b in b_levels:
                group = data[(data[factor_a] == a) & (data[factor_b] == b)][value_col]
                if len(group) == 0:
                    continue
                mean_ab = group.mean()
                mean_a = data[data[factor_a] == a][value_col].mean()
                mean_b = data[data[factor_b] == b][value_col].mean()
                SSAB += len(group) * ((mean_ab - mean_a - mean_b + grand_mean) ** 2)

        # SS Within/Error
        SSW = 0
        for a in a_levels:
            for b in b_levels:
                group = data[(data[factor_a] == a) & (data[factor_b] == b)][value_col]
                if len(group) == 0:
                    continue
                mean_group = group.mean()
                SSW += ((group - mean_group) ** 2).sum()

        # Mean Squares
        MSA = SSA / dfA
        MSB = SSB / dfB
        MSAB = SSAB / dfAB
        MSW = SSW / dfW

        # F-statistics
        FA = MSA / MSW
        FB = MSB / MSW
        FAB = MSAB / MSW

        # p-values
        pA = 1 - f.cdf(FA, dfA, dfW)
        pB = 1 - f.cdf(FB, dfB, dfW)
        pAB = 1 - f.cdf(FAB, dfAB, dfW)

        self.results = {
            'Factor A': {'SS': SSA, 'DF': dfA, 'MS': MSA, 'F': FA, 'p': pA},
            'Factor B': {'SS': SSB, 'DF': dfB, 'MS': MSB, 'F': FB, 'p': pB},
            'Interaction': {'SS': SSAB, 'DF': dfAB, 'MS': MSAB, 'F': FAB, 'p': pAB},
            'Error': {'SS': SSW, 'DF': dfW, 'MS': MSW}
        }

    def summary(self):
        print(f"{'Source':<12} {'SS':>10} {'DF':>5} {'MS':>10} {'F':>10} {'p-value':>10}")
        print("-" * 60)
        for k, v in self.results.items():
            line = f"{k:<12} {v['SS']:>10.2f} {v['DF']:>5} {v['MS']:>10.2f}"
            if 'F' in v:
                line += f" {v['F']:>10.2f} {v['p']:>10.4f}"
            print(line)


In [18]:
import pandas as pd

In [149]:
df = pd.read_excel(r"C:\Users\saqli\Desktop\FULL DataSet.xlsx")

In [150]:
df.head()

,Treatment,Alginate,Guargum,Pectin,Days,Weight_loss,Firmness,TSS,pH,L*,a*,b*,delta_E,TPC,DPPH
0,0,0.0,0.0,0.0,0,0.00000,721.00,7.90,4.12,19.62,9.51,21.46,0.000000,510.83,67.76
1,0,0.0,0.0,0.0,2,18.19776,468.69,4.76,4.11,15.48,6.85,18.87,6.508748,716.88,123.20
2,1,3.0,0.0,0.0,2,18.43008,500.96,4.85,4.07,16.47,7.51,19.52,4.172649,498.13,120.10
3,2,0.0,3.0,0.0,2,18.33792,513.90,4.80,4.08,15.93,6.98,19.46,5.339110,436.88,161.50
4,3,0.0,0.0,3.0,2,18.47232,591.24,5.04,4.11,17.41,8.42,19.81,2.965586,413.13,162.30


In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Treatment    34 non-null     int64  
 1   Alginate     34 non-null     float64
 2   Guargum      34 non-null     float64
 3   Pectin       34 non-null     float64
 4   Days         34 non-null     int64  
 5   Weight_loss  34 non-null     float64
 6   Firmness     34 non-null     float64
 7   TSS          34 non-null     float64
 8   pH           34 non-null     float64
 9   L*           34 non-null     float64
 10  a*           34 non-null     float64
 11  b*           34 non-null     float64
 12  delta_E      34 non-null     float64
 13  TPC          34 non-null     float64
 14  DPPH         34 non-null     float64
dtypes: float64(13), int64(2)
memory usage: 4.1 KB


In [153]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Treatment    34 non-null     int64  
 1   Alginate     34 non-null     float64
 2   Guargum      34 non-null     float64
 3   Pectin       34 non-null     float64
 4   Days         34 non-null     float64
 5   Weight_loss  34 non-null     float64
 6   Firmness     34 non-null     float64
 7   TSS          34 non-null     float64
 8   pH           34 non-null     float64
 9   L*           34 non-null     float64
 10  a*           34 non-null     float64
 11  b*           34 non-null     float64
 12  delta_E      34 non-null     float64
 13  TPC          34 non-null     float64
 14  DPPH         34 non-null     float64
dtypes: float64(14), int64(1)
memory usage: 4.1 KB


In [ ]:
model = TwoWayAnova()
model.fit(df_new, 'TPC', 'Treatment', 'Days')
model.summary()

Source               SS    DF         MS          F    p-value
------------------------------------------------------------
Factor A       73574.10    10    7357.41   31184.27     0.0000
Factor B      262005.43     3   87335.14  370168.72     0.0000
Interaction  1259181.76    30   41972.73  177900.78     0.0000
Error              5.66    24       0.24


In [158]:
model = TwoWayAnova()
model.fit(df_new, 'delta_E', 'Days', 'Treatment')
model.summary()

Source               SS    DF         MS          F    p-value
------------------------------------------------------------
Factor A         314.70     3     104.90     220.82     0.0000
Factor B          59.38    10       5.94      12.50     0.0000
Interaction      113.69    30       3.79       7.98     0.0000
Error             27.55    58       0.48


In [126]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

df_clean = df.dropna()

model = ols('TSS ~ C(Treatment) + C(Days)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)


                 sum_sq    df          F        PR(>F)
C(Treatment)   9.084000  10.0   2.933695  1.941712e-02
C(Days)       42.198936   3.0  45.427421  4.097848e-09
Residual       6.192873  20.0        NaN           NaN
